In [1]:
import numpy as np
from numpy.linalg import inv
from pymor.basic import *
from pymor.reductors.basic import StationaryRBReductor
from pymor.algorithms.to_matrix import to_matrix
from pymor.algorithms.gram_schmidt import gram_schmidt 
from pymor.models.basic import StationaryModel
from pymor.operators.constructions import LincombOperator
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.parameters.functionals import ProjectionParameterFunctional, ConjugateParameterFunctional
from pymor.vectorarrays.numpy import NumpyVectorSpace

In [2]:
def MatrixModel(A, B, C):
    '''
    This function creates stationary models for the following linear coercive models derived for the given three matrices A, B, and C:
    
        a_1(v, w; s) = w*(sI_n - A)v and l_1(w) = w*B
        a_2(v, w; s) = w*(sI_n - A)*v and l_2(w) = w*C^T.

    Inputs:
    ------------------------------------------------
    A - matrix -> NumPy array or NumpyMatrixOperator
    B - vector -> NumPy array or NumpyMatrixOperator
    C - vector -> NumPy array or NumpyMatrixOperator
    ------------------------------------------------
    Outputs:
    ------------------------------------------------
    model_V - Stationary Model of linear coercive model w*(sI_n - A)v = w*B -> StationaryModel
    model_W - Stationary Model of linear coercive model w*(sI_n - A)*v -> StationaryModel
    '''

    # Define operators (and also a dimension of a model)
    if isinstance(A, np.ndarray):
        dim = A.shape[0]
        A_op = NumpyMatrixOperator(A)
    else:
        dim = to_matrix(A).shape[0]
        A_op = A

    if isinstance(B, np.ndarray):
        B_op = NumpyMatrixOperator(B) 
    else:
        B_op = B  

    if isinstance(C, np.ndarray):
        C_op = NumpyMatrixOperator(C.T)  
    else:
        C_op = C.H  # C is real, so adjoint is transpose

    I_op = NumpyMatrixOperator(np.eye(dim))
    
    # Define parameter functional for 's'
    s_param = ProjectionParameterFunctional('s', 1)

    # Define bilinear form a(v, w; s) = w*(sI - A)v
    a_op_1 = LincombOperator([I_op, A_op], [s_param, -1])

    # Define bilinear form a(v, w; s) = w*(sI - A)*v -> Note: (sI - A)* = s*I - A*
    a_op_2 = LincombOperator([I_op, A_op.H], [ConjugateParameterFunctional(s_param), -1])
    
    # Define linear functional l(w) = w^*B
    l_op_1 = B_op

    # Define linear functional l(w) = w^*C^{T}
    l_op_2 = C_op

    # Define the StationaryModels
    model_V = StationaryModel(operator=a_op_1, rhs=l_op_1)
    model_W = StationaryModel(operator=a_op_2, rhs=l_op_2)

    return [model_V, model_W] 


In [3]:
def MatrixReductor(model_V, model_W, training_set, reduced_order_V: int, reduced_order_W: int):
    
    '''
    Inputs:
    ------------------------------------------------
    model_V - Stationary Model of linear coercive model w*(sI_n - A)v = w*B -> StationaryModel
    model_W - Stationary Model of linear coercive model w*(sI_n - A)*v -> StationaryModel
    training_set - an array containing parameters used to construct the snapshot matrix -> type(training_set[i]) = pymor.parameters.base.Mu (list of Mu objects)
    ------------------------------------------------
    Outputs:
    ------------------------------------------------
    pod_rom_V
    pod_rom_W
    '''

    # Compute FOM solutions for the parameters in the training set
    solution_snapshots_V = model_V.solution_space.empty()
    solution_snapshots_W = model_W.solution_space.empty()
    for s in training_set:
        solution_snapshots_V.append(model_V.solve(s))
        solution_snapshots_W.append(model_W.solve(s))
        
    # Snapshot matrices
    snapshot_matrix_V = solution_snapshots_V.to_numpy().T # Note: One may also use solution_snapshots_V.impl._array.T to get np.ndarray type needed for computation
    snapshot_matrix_W = solution_snapshots_W.to_numpy().T

    # Finding the Singular Value Decomposition (SVD) of snapshot matrices -> S = UΣV^T
    U_V, D_V, Vt_V = np.linalg.svd(snapshot_matrix_V, full_matrices = True)
    U_W, D_W, Vt_W = np.linalg.svd(snapshot_matrix_W, full_matrices = True)

    if reduced_order_V > min(snapshot_matrix_V.shape):
        raise ValueError("'reduced_order_V' cannot exceed the rank of the snapshot matrix.")
    if reduced_order_W > min(snapshot_matrix_W.shape):
        raise ValueError("'reduced_order_W' cannot exceed the rank of the snapshot matrix.")

    # The reduced bases (POD bases)
    pod_basis_numpy_V = U_V[:,:reduced_order_V]
    pod_basis_numpy_W = U_W[:,:reduced_order_W]

    # Convert NumPy array into VectorArray 
    space_V = NumpyVectorSpace(model_V.order) #number of columns = model_V.order
    space_W = NumpyVectorSpace(model_W.order)
    pod_basis_V = space_V.make_array(pod_basis_numpy_V.T) #This is actually transpose of POD-RB basis
    pod_basis_W = space_W.make_array(pod_basis_numpy_W.T) #This is actually transpose of POD-RB basis
    
    # POD-Galerkin RB method
    pod_reductor_V = StationaryRBReductor(model_V, RB = pod_basis_V) 
    pod_reductor_W = StationaryRBReductor(model_W, RB = pod_basis_W) 
    pod_rom_V = pod_reductor_V.reduce()
    pod_rom_W = pod_reductor_W.reduce()

    return [pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W]


In [4]:
def ProjectionMatrices(pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W, mu, b, c, numpy = False):

    '''
    Inputs:
    ------------------------------------------------
    pod_rom_V
    pod_rom_W
    pod_reductor_V
    pod_reductor_W
    mu - list of -mu_i values -> type(mu[i]) = pymor.parameters.base.Mu (list of Mu objects)
    b - NumPy array -> b.shape = (r,) where r = len(mu)
    c - NumPy array -> c.shape = (r,) where r = len(mu)
    validation_set - an array containing parameters used to evaluate the reduced model after its construction -> type(validation_set[i]) = pymor.parameters.base.Mu (list of Mu objects)
    ------------------------------------------------
    Outputs: Biorthonormal pair of projection matrices V, W using biorthonormal Gram-Schmidt process
    ------------------------------------------------
    V - projection matrix V -> NumpyVectorArray -> V.shape = (n, r)
    W - projection matrix W -> NumpyVectorArray -> W.shape = (n, r)
    '''
    
    # Solution arrays containing len(validation_set) many reduced samples
    card_mu = len(mu)
    reduced_solution_V = pod_rom_V.solution_space.empty()
    reduced_solution_W = pod_rom_W.solution_space.empty()
    for s in mu:
        reduced_solution_V.append(pod_rom_V.solve(s))
        reduced_solution_W.append(pod_rom_W.solve(s))
        
    # It would be better to get matrices where columns are the reconstructed reduced solutions as in theory we will use such matrix; however PyMor only has vstack option (appending as a row of a matrix)
    reduced_solution_reconstruct_V_T = pod_reductor_V.reconstruct(reduced_solution_V) # a matrix with rows representing the reconstructed reduced solutions for different parameter values to first parametrized coercive model (row i will give us (s_{i}I - A)^{-1}B)
    reduced_solution_reconstruct_W_T = pod_reductor_W.reconstruct(reduced_solution_W) # a matrix with rows representing the reconstructed reduced solutions for different parameter values to second parametrized coercive model (row i will give us (s_{i}I - A)^{-*}C^T)

    # To align with the theory, we take the transpose of the result. Also, note that the transpose operation does not exist in PyMor for `NumpyVectorArray`, so we first take the transpose of the NumPy array and then convert it back
    space_V_numpy = NumpyVectorSpace(card_mu)
    space_W_numpy = NumpyVectorSpace(card_mu)
    R_V = space_V_numpy.make_array(reduced_solution_reconstruct_V_T.to_numpy().T)
    R_W = space_W_numpy.make_array(reduced_solution_reconstruct_W_T.to_numpy().T)

    R_V, R_W = R_V.to_numpy(), R_W.to_numpy() # Note: One may also use R_V.impl._array to get np.ndarray type needed for computation; also above convertion is unnnecessary one may remove it
    D_b, D_c = np.diag(b), np.diag(c)

    V_numpy = np.matmul(R_V, D_b)
    W_numpy = np.matmul(R_W, D_c)

    if numpy is True: # exact values
        V = V_numpy
        W = W_numpy
    else: # to work in pymor everything is transposed
        space = NumpyVectorSpace(V_numpy.shape[0])
        V = space.make_array(V_numpy.T)
        W = space.make_array(W_numpy.T)

    return [V, W]

In [5]:
def Gram(V, W, biort = False, numpy = False):

    [Q_V, _] = gram_schmidt(V, return_R = True, atol = 0, rtol = 0, check_tol=1e-10)
    [Q_W, _] = gram_schmidt(W, return_R = True, atol = 0, rtol = 0, check_tol=1e-10)
    V_orth = Q_V.to_numpy().T
    W_orth = Q_W.to_numpy().T

    if biort is True:
        M = np.linalg.inv(W_orth.T@V_orth)
        V_orth = V_orth@M
        if numpy is True:
            V = V_orth
            W = W_orth
        else:
            space = NumpyVectorSpace(V_orth.shape[0])
            V = space.make_array(V_orth.T)
            W = space.make_array(W_orth.T)
    else:
        if numpy is True:
            V = V_orth
            W = W_orth
        else:
            V = Q_V
            W = Q_W
    return [V, W]

In [6]:
def exact(A, B, C, mu, b, c, numpy = False):

    dim = A.shape[0] if isinstance(A, np.ndarray) else to_matrix(A).shape[0]
    
    # NumPy convertions
    A = A if isinstance(A, np.ndarray) else to_matrix(A).toarray()
    B = B if isinstance(B, np.ndarray) else to_matrix(B)
    C = C if isinstance(C, np.ndarray) else to_matrix(C)
    mu_values = np.array([s['s'] for s in mu]) 
    
    # Exact projection matrices
    identity = np.eye(dim)
    D_b, D_c = np.diag(b), np.diag(c)
    
    V_exact = np.matmul(np.hstack([np.matmul(inv(s*identity - A), B) for s in mu_values]), D_b)
    W_exact = np.matmul(np.hstack([(np.matmul(np.conjugate(inv(s*identity - A).T), C.T)) for s in mu_values]), D_c)

    if numpy is True:
        V = V_exact
        W = W_exact
    else: # transposed version needed for Gram process
        space = NumpyVectorSpace(V_exact.shape[0])
        V = space.make_array(V_exact.T)
        W = space.make_array(W_exact.T)
    return [V, W]

In [7]:
def error(V_exact, W_exact, V, W, norm):

    # NumPy convertions
    V_exact = V_exact if isinstance(V_exact, np.ndarray) else V_exact.to_numpy()
    W_exact = W_exact if isinstance(W_exact, np.ndarray) else W_exact.to_numpy()
    V = V if isinstance(V, np.ndarray) else V.to_numpy()
    W = W if isinstance(W, np.ndarray) else W.to_numpy()

    error_V = np.linalg.norm(V_exact - V, norm)
    error_W = np.linalg.norm(W_exact - W, norm)

    print(f'V_exact - V is {error_V} and W_exact - W is {error_W}')

In [8]:
# Example

from pymor.models.examples import penzl_example

penzl = penzl_example()

A = penzl.A
B = penzl.B
C = penzl.C

[model_V, model_W] = MatrixModel(A, B, C)

# Define a parameter space
parameter_space = model_V.parameters.space(0.01, 10.)

# Define a training set
training_set = parameter_space.sample_randomly(100)

# Define interpolation data
b = np.random.rand(100)
c = np.random.rand(100)
mu = parameter_space.sample_randomly(100)

In [9]:
# Create POD-Galerkin RB reductors
[pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W] = MatrixReductor(model_V, model_W, training_set, reduced_order_V = 100, reduced_order_W = 100)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [10]:
# Get projection matrices before Gram process
[V, W] = ProjectionMatrices(pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W, mu, b, c, numpy = False)

In [11]:
# Get exact values for projection matrices before Gram process
[V_exact, W_exact] = exact(A, B, C, mu, b, c, numpy = False)

In [12]:
# Get projection matrices after Gram process
[V_g, W_g] = Gram(V, W, biort = False, numpy = False)
[V_gbi, W_gbi] = Gram(V, W, biort = True, numpy = False)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [13]:
# Get exact projection matrices after Gram process
[V_exact_g, W_exact_g] = Gram(V_exact, W_exact, biort = False, numpy = False)
[V_exact_gbi, W_exact_gbi] = Gram(V_exact, W_exact, biort = True, numpy = False)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [21]:
pod_error = error(V_exact, W_exact, V, W, np.inf)
pod_error_g = error(V_exact_g, W_exact_g, V_g, W_g, np.inf)
pod_error_gbi = error(V_exact_gbi, W_exact_gbi, V_gbi, W_gbi, np.inf)

V_exact - V is 1.6655188513070573e-14 and W_exact - W is 2.319097604924769e-14
V_exact - V is 30.83508408465606 and W_exact - W is 30.1557504081609
V_exact - V is 23311.6444169422 and W_exact - W is 30.1557504081609


In [34]:
np.sum((V_exact_gbi.to_numpy().T - V_gbi.to_numpy().T) >=240)

np.int64(0)